In [ ]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import string
import re
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Embedding, LSTM, Dropout, Activation, Dense

In [ ]:
df = pd.read_csv("../input/preallemails/data.csv")
df

In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=500)
tokenizer.fit_on_texts(df["text"])
matrix = tokenizer.texts_to_matrix(df["text"])

In [ ]:
def RNN():
    inputs = Input(name="inputs", shape=[500])
    layer = Embedding(200, 100, input_length=500)(inputs)
    layer = LSTM(64)(layer)
    layer = Activation("relu")(layer)
    layer = Dropout(0.4)(layer)
    layer = Dense(128, name="FC1")(layer)
    layer = Activation("relu")(layer)
    layer = Dense(1, name="out_layer")(layer)
    layer = Activation("sigmoid")(layer)
    model = Model(inputs=inputs, outputs=layer)
    model.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

In [ ]:
tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.config.experimental_connect_to_cluster(tpu)
tf.tpu.experimental.initialize_tpu_system(tpu)

tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


with tpu_strategy.scope():
    model = RNN()    

In [ ]:
from sklearn.model_selection import train_test_split
Xtrain, Xtest, Ytrain, Ytest = train_test_split(matrix, df["label"], test_size=0.33)

In [ ]:
train_acc = []
evaluate_acc = []
for i in range(500):
    print(f"epoch: {i+1}")
    acc = model.fit(Xtrain, Ytrain, epochs= 1)
    
    train_acc.append(acc.history["accuracy"][0])
    eacc = model.evaluate(Xtest, Ytest)
    if len(evaluate_acc) > 0 and max(evaluate_acc) < eacc[1]:
        model.save(f"./{i+1}.h5")
    evaluate_acc.append(eacc[1])

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(train_acc)

In [ ]:
plt.plot(evaluate_acc)